In [1]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split

In [2]:
# pip install -q transformers

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 564.3/564.3 kB 10.1 MB/s eta 0:00:00 0:00:01
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
datasets 4.1.1 requires pyarrow>=21.0.0, but you have pyarrow 19.0.1 which is incompatible.
gradio 5.38.1 requires pydantic<2.12,>=2.0, but you have pydantic 2.12.0a1 which is incompatible.
Note: you may need to restart the kernel to use updated packages.


In [3]:
dataset = pd.read_csv("/kaggle/input/postagger/POS.csv")

In [4]:
import datasets
datasets.__version__

'4.1.1'

In [5]:
dataset.head()

,sentence_id,words,labels
0,0,﻿६१,CD
1,0,वर्षीय,JJ
2,0,पियरे,NNP
3,0,भिन्केन,NNP
4,0,नोभेम्बर,NNP


In [6]:
label_list = dataset['labels'].unique().tolist()
print(len(label_list))

39


In [7]:
label2ids = {label:i for i,label in enumerate(label_list)}
ids2label = {values:keys for keys,values in label2ids.items()}

In [8]:
print(label2ids)
print(ids2label)

{'CD': 0, 'JJ': 1, 'NNP': 2, 'POP': 3, 'NN': 4, 'PKO': 5, 'VBX': 6, 'YF': 7, 'FB': 8, 'VBF': 9, 'PLAI': 10, 'DUM': 11, 'VBKO': 12, 'RBO': 13, 'VBI': 14, 'VBO': 15, 'HRU': 16, 'JJD': 17, 'YM': 18, 'PLE': 19, 'JJM': 20, 'RP': 21, 'VBNE': 22, 'CS': 23, 'YQ': 24, 'CL': 25, 'PP': 26, 'PP$': 27, 'CC': 28, 'SYM': 29, 'PPR': 30, 'DM': 31, 'OD': 32, 'QW': 33, 'UNW': 34, 'RBM': 35, 'FW': 36, 'YB': 37, 'ALPH': 38}
{0: 'CD', 1: 'JJ', 2: 'NNP', 3: 'POP', 4: 'NN', 5: 'PKO', 6: 'VBX', 7: 'YF', 8: 'FB', 9: 'VBF', 10: 'PLAI', 11: 'DUM', 12: 'VBKO', 13: 'RBO', 14: 'VBI', 15: 'VBO', 16: 'HRU', 17: 'JJD', 18: 'YM', 19: 'PLE', 20: 'JJM', 21: 'RP', 22: 'VBNE', 23: 'CS', 24: 'YQ', 25: 'CL', 26: 'PP', 27: 'PP$', 28: 'CC', 29: 'SYM', 30: 'PPR', 31: 'DM', 32: 'OD', 33: 'QW', 34: 'UNW', 35: 'RBM', 36: 'FW', 37: 'YB', 38: 'ALPH'}


In [9]:
dataset.isna().sum()

sentence_id    0
words          2
labels         0
dtype: int64

In [10]:
index = dataset[dataset["words"].isna()].index.tolist()
print(index)

[61652, 73919]


In [11]:
dataset = dataset.drop(index=index)

In [12]:
print(len(dataset["sentence_id"].unique()))

4251


In [13]:
grouped = dataset.groupby("sentence_id")
print(len(grouped["words"]))

4251


In [14]:
df_grouped = grouped.agg({
    "words":lambda x:list(x),
    "labels":lambda x:list(x)
})

In [15]:
df_grouped.columns

Index(['words', 'labels'], dtype='object')

In [16]:
train,test = train_test_split(
    df_grouped,
    test_size = 0.2,
    shuffle = True
)

In [17]:
train.reset_index(drop=True,inplace=True)

In [18]:
train.head()

,words,labels
0,"[इन्टरपब्लिक, ले, विज्ञापन, समय, को, सट्टा, मा...","[NN, PLE, NN, NN, PKO, NN, POP, NN, HRU, JJ, V..."
1,"[न्यायिक, विभाग, को, नीति, सँग, परिचित, व्यक्त...","[JJ, NN, PKO, NN, POP, JJ, NN, HRU, PLE, VBF, ..."
2,"[श्री, रिज, भन्नु, हुन्छ, अमेरिकी, एक्सप्रेस, ...","[NN, NNP, VBI, VBF, JJ, NN, PLE, NN, NN, CC, N..."
3,"[कमनवेल्थ, एडिसन, का, एक, प्रवक्ता, ले, गत, १/...","[NNP, NNP, PKO, CD, NN, PLE, JJ, CD, NN, POP, ..."
4,"[वास्तविक, लडाइँ, त, कस, ले, त्यो, बजार, नियन्...","[JJ, NN, RP, DUM, PLE, DUM, NN, NN, VBNE, CC, ..."


In [19]:
from transformers import AutoTokenizer

tokenizer = AutoTokenizer.from_pretrained("Shushant/nepaliBERT")


config.json:   0%|          | 0.00/589 [00:00<?, ?B/s]

vocab.txt: 0.00B [00:00, ?B/s]

In [20]:
print(train['words'].dtype)


object


In [21]:
print(type(train["words"]))

<class 'pandas.core.series.Series'>


In [22]:
tokenized_input = tokenizer(train.iloc[0]["words"],is_split_into_words=True)

In [23]:
print(tokenized_input.keys())

KeysView({'input_ids': [2, 6274, 2054, 1031, 2746, 1706, 396, 2982, 1107, 1005, 2377, 2003, 1805, 15477, 1811, 1723, 2404, 2408, 1007, 1028, 2699, 2723, 1013, 1064, 5454, 1832, 1672, 373, 16, 2366, 1805, 6056, 24072, 3992, 13420, 1723, 4647, 11927, 4774, 8198, 1743, 2243, 394, 25111, 1723, 11822, 11927, 4774, 8198, 1743, 2243, 402, 1030, 1782, 396, 16372, 428, 4], 'token_type_ids': [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0], 'attention_mask': [1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1]})


In [24]:
tokens = tokenizer.convert_ids_to_tokens(tokenized_input['input_ids'])

In [25]:
word_ids = tokenized_input.word_ids(batch_index=0) 
print(word_ids)

[None, 0, 0, 0, 0, 0, 1, 2, 2, 2, 2, 3, 4, 5, 5, 6, 7, 7, 7, 7, 8, 9, 9, 9, 10, 10, 10, 11, 12, 13, 14, 15, 15, 15, 16, 17, 18, 19, 20, 21, 21, 22, 23, 24, 25, 26, 27, 28, 29, 29, 30, 31, 31, 32, 33, 34, 35, None]


In [26]:
print(tokens)

['[CLS]', 'इन', '##टर', '##प', '##बल', '##िक', 'ल', 'विज', '##ञ', '##ा', '##पन', 'समय', 'को', 'सट', '##टा', 'मा', 'कार', '##यक', '##र', '##म', 'हर', 'उपल', '##ब', '##ध', 'गराइ', '##रह', '##को', 'छ', ',', 'जस', 'को', 'मल', '##याङ', '##कन', '१९९०', 'मा', '७५', 'मिलियन', 'डलर', 'भन', '##दा', 'बढी', 'र', '१९९१', 'मा', '१५०', 'मिलियन', 'डलर', 'भन', '##दा', 'बढी', 'ह', '##न', 'यस', 'ल', 'बतायो', '।', '[SEP]']


In [27]:
from datasets import Dataset
dataset_train = Dataset.from_pandas(train)
dataset_test = Dataset.from_pandas(test)

In [28]:
print(type(dataset_train['words']))              # datasets.arrow_dataset.Column
print(type(dataset_train['words'][0]))           # list
print(type(dataset_train['words'][0][0]))        # str
print(type(dataset_train['labels'][0]))          # list
print(type(dataset_train['labels'][0][0]))       # str or int depending on mapping


<class 'datasets.arrow_dataset.Column'>
<class 'list'>
<class 'str'>
<class 'list'>
<class 'str'>


In [29]:
tokenized_input = tokenizer(dataset_train["words"][:100],is_split_into_words=True)

In [30]:
# print(dataset_train['words'][0])
# for i,data in enumerate(dataset_train['labels'][0]):
#     print(f"{i}:{data}")
#     print(label2ids[data])
    
 

In [31]:
def tokenize_and_align_labels(examples):
    tokenize_inputs = tokenizer(examples['words'][:],truncation=True,max_length = 512,is_split_into_words=True)
    labels = []

    for i,label in enumerate(examples['labels'][:]):
        word_ids = tokenize_inputs.word_ids(batch_index=i)
        previous_word_idx = None
        label_ids = []
        for word_idx in word_ids:
            if word_idx is None:
                label_ids.append(-100)
            elif word_idx != previous_word_idx:
                label_ids.append(label2ids[label[word_idx]])
            else:
                label_ids.append(-100)
            previous_word_idx = word_idx
        labels.append(label_ids)

    tokenize_inputs['labels'] = labels
    return tokenize_inputs
                
        
    

In [32]:

tokenized_inputs = dataset_train.map(tokenize_and_align_labels,batched=True,batch_size=100)

Map:   0%|          | 0/3400 [00:00<?, ? examples/s]